In [1]:
import numpy as np
import scipy as sp
import time
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/gmr-1.1_git-py2.7.egg')
import gmr as gaussian_regression

In [2]:
#Environment definition
from scipy import spatial
from explauto.utils import rand_bounds
from explauto.environment.environment import Environment
from explauto import SensorimotorModel
from explauto.sensorimotor_model import sensorimotor_models

In [3]:
from explauto.models.gmrinf import GMR
from explauto.sensorimotor_model.ilo_gmr import IloGmr

In [4]:
environment = Environment.from_configuration('simple_arm', 'mid_dimensional')

In [5]:
sm_cls, sm_configs = sensorimotor_models['nearest_neighbor']
model = sm_cls(environment.conf, **sm_configs['default'])

In [6]:
#Creation of the total dataset
def generateFwdDataset(_environment, _model, _n = 1000):
    random_motors = _environment.random_motors(n=_n)
   
    for m in random_motors:
        s = _environment.compute_sensori_effect(m)
        _model.update(m, s)
generateFwdDataset(environment, model)

In [7]:
data_list = model.model.imodel.fmodel.dataset.data

In [8]:
data_np = np.array(data_list).T

In [9]:
data = []
for i in range(len(data_np)):
    data.append(np.hstack(data_np[i,:]))

In [10]:
gmr = GMR(n_components=3)
gmr = gmr.from_samples(np.array(data))

In [11]:
x_wanted_list = [-0.76151596,  0.85728845,  0.0987334 ,  0.46247676, -0.16366163,
         0.27212757,  0.08281898]
x_wanted_array = np.transpose(np.array([[-0.76151596],  [0.85728845],  [0.0987334],  [0.46247676], [-0.16366163],
         [0.27212757],  [0.08281898]]))

In [12]:
conditional = gmr.condition([0, 1, 2, 3, 4, 5, 6], x_wanted )

In [13]:
print conditional.priors

[  1.37649826e-04   2.95463259e-02   9.70316024e-01]


In [15]:
prediction = gmr.predict(np.array([0, 1, 2, 3, 4, 5, 6]), x_wanted_array)


In [16]:
print prediction[0]

[ 0.83352573 -0.06368539]


In [18]:
reg = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]), x_wanted_array, "lse")

In [19]:
print reg

[ 0.83352573 -0.06368539]


In [23]:
slse = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]),  x_wanted_array, "slse")

In [21]:
print slse

[ 0.82906377 -0.07093158]


In [22]:
stoc_samp = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]),  x_wanted_array, "stochastic_sampling")

In [24]:
print stoc_samp
print conditional.sample(n_samples=1)[0]

[ 0.79811269 -0.01681426]
[ 0.80222399 -0.03754475]


In [22]:
optim = gmr.regression(np.array([0, 1, 2, 3, 4, 5, 6]), np.array([7,8]),  x_wanted_array, "optimization")

TypeError: regression_optimization() takes at least 5 arguments (4 given)

In [12]:
#1) get x the nearest neighbor of xq in the dataset
dist, indexes = model.model.imodel.fmodel.dataset.nn_x( x_wanted_list, k = 1)
x = model.model.imodel.fmodel.dataset.get_x(indexes[0])
print x_wanted_list
print x

[-0.76151596, 0.85728845, 0.0987334, 0.46247676, -0.16366163, 0.27212757, 0.08281898]
[-0.46137381  0.43977351  0.18866761  0.39337473  0.04888891  0.60912177
  0.15361874]


In [13]:
#2) get the y associated to x
y = model.model.imodel.fmodel.dataset.get_y(indexes[0])
print y

[ 0.87630398  0.01291787]


In [14]:
#3) Creation fof a local dataset depending on x_g
_, l_index = model.model.imodel.fmodel.dataset.nn_x(x, 100)
l_nn = []
col1 = []
col2 = []
for index in l_index:
    l_nn.append(np.hstack(model.model.imodel.fmodel.dataset.get_xy(index)))
lnn_nparray = np.array(l_nn)

In [40]:
#4) find the number of gaussians
from sklearn.cluster import AffinityPropagation
af = AffinityPropagation(damping=0.988)
af.fit(l_nn)
n_clusters = len(af.cluster_centers_)
print n_clusters

3


In [41]:
#5 ) create ilogmr
local_gmr = IloGmr(environment.conf, n_clusters)
local_gmr = local_gmr.from_samples(lnn_nparray)

In [42]:
print local_gmr.priors

[ 0.26908044  0.46998805  0.26093151]
